In [ ]:
import numpy as np 
import pandas as pd
from scipy import stats 
import matplotlib.pyplot as plt
from ipynb.fs.full.Quantile_Normalization import quantile_norm, quantile_norm_log

# Import TCGA melanoma data 
## Rna read data
file='../data/counts.txt'
with open(file, 'rt') as f: 
    read_counts=pd.read_csv(f,index_col=0) 
    
counts=read_counts.values

In [ ]:
log_counts_normalized=quantile_norm_log(counts)

In [ ]:
def most_variable_rows(data,*,n=1500):
    """
    
    Subset n most variable genes
    
    Parameters
    ..........
    
    data: 2D array of float
    n: int, optional 
        Number of rows to return. 
    
    Return
    ..........
    variable_data=2D array of float 
    
    """
    
    # compute accorss columns
    
    rowvar=np.var(data,axis=1)
    # get indices
    
    sort_indices=np.argsort(rowvar)[-n:]
    
    variable_data=data[sort_indices,:]
    
    return variable_data



In [ ]:
from scipy.cluster.hierarchy import linkage

def bicluster(data, linkage_method='average',distance_metric='correlation'):
    """
    
    Subset n most variable genes
    
    Parameters
    ..........
    
    data: 2D array of float
        the input data to bicluster
    linkage_method: string, optional 
        method to be passed linkage 
    distance_metric:
        distance metric for clustering 
    
    Returns
    ..........
        
    y_rows: linkage_matrix
        The clustering of the rows of the input data 
    y_cols:linkage matrix 
        The clustering of the cols of the input data

    """
    y_rows=linkage(data,method=linkage_method,metric=distance_metric)
    y_cols=linkage(data.T,method=linkage_method,metric=distance_metric)
    return y_rows,y_cols


In [ ]:
from scipy.cluster.hierarchy import dendrogram, leaves_list

def clear_spines(axes):
    for loc in ['left','right','top','bottom']:
        axes.spines[loc].set_visible(False)
    axes.set_xticks([])
    axes.set_yticks([])

def plot_bicluster(data,row_linkage,col_linkage,row_nclusters=10,col_nclusters=3):
    """
    
    Perform a biclusetring, plot a heatmap with dendrograms on each axis
    
    
    Parameters 
    ..........
    
    data: 2D array of float shape (m,n)
    row_linkage:array, shape (m-1,4)
        linkage matrix for rows of data
    col_linkage: array shape (n-1,4)
        the linakge matrix for the columns of 'data'
    n_clusters, n_clusters_c: int, optional
        number of clusters for rows and colums
    
    Returns 
    ..........
    
    Xn: 2D array of float shape (m,n)
        data normalized
    
    """
    
    fig=plt.figure(figsize=(15,15))
    
    ## compute row-wsie dendrogram
    
    ax1=fig.add_axes([0.09,0.1,0.2,0.6])
    
    threshold_r=(row_linkage[-row_nclusters,2] + row_linkage[-row_nclusters + 1,2])/2
    with plt.rc_context({'lines.linewidth':0.75}):
        dendrogram(row_linkage,orientation='left',color_threshold=threshold_r,ax=ax1)
        
    clear_spines(ax1)
    
    ax2=fig.add_axes([0.3,0.71,0.6,0.2])
    
    threshold_c=(col_linkage[-col_nclusters,2] + col_linkage[-col_nclusters + 1,2])/2
    
    with plt.rc_context({'lines.linewidth':0.75}):
        dendrogram(col_linkage,color_threshold=threshold_c,ax=ax2)
        
    clear_spines(ax2)
    
    ax=fig.add_axes([0.3,0.1,0.6,0.6])
    
    idx_rows=leaves_list(row_linkage)
    data=data[idx_rows,:]
    idx_cols=leaves_list(col_linkage)
    data=data[:,idx_cols]
    
    im=ax.imshow(data,aspect='auto',origin='lower',cmap='YlGnBu_r')
    clear_spines(ax)
    
    ax.set_xlabel('Samples')
    ax.set_ylabel('Genes',labelpad=125)
    
    axcolor=fig.add_axes([.9,.1,.02, .6])
    plt.colorbar(im,cax=axcolor)
    
    plt.show()

In [ ]:
counts_log=log_counts_normalized
counts_var=most_variable_rows(counts_log,n=1500)
yr,yc=bicluster(counts_var,linkage_method='ward',distance_metric='euclidean')
with plt.style.context('ggplot'):
    plot_bicluster(counts_var,yr,yc)